In [19]:
import os

In [20]:
os.environ["MLFLOW_TRACKING_URI"] = 'http://localhost:8990'
os.environ["MLFLOW_EXPERIMENT_NAME"] = "Prueba_local"
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

In [21]:
os.environ["MLFLOW_TRACKING_URI"]

'http://localhost:8990'

In [22]:
import mlflow
import pandas as pd
import numpy as np
import mlflow.pyfunc
from sklearn.model_selection import train_test_split

In [23]:
database_dir = 'c:\\database\\winedata\\'
file_name = 'winequality-red.csv'

In [24]:
data = pd.read_csv(database_dir + file_name, sep = ';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [25]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [26]:
# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [27]:
model_name = 'ClasificadorDeVinos'
version = "3"

model_version_uri = "models:/{model_name}/{version}".format(model_name=model_name, version=version)
model_version = mlflow.pyfunc.load_model(model_version_uri)
model_version

mlflow.pyfunc.loaded_model:
  artifact_path: PlainRegression_Model
  flavor: mlflow.sklearn
  run_id: 0d8fac9bb8554b0197db9c06bafbe826

In [28]:
model_name = 'ClasificadorDeVinos'
area = 'staging'

model_staging_uri = "models:/{model_name}/{area}".format(model_name=model_name, area=area)
model_staging = mlflow.pyfunc.load_model(model_staging_uri)
model_staging

mlflow.pyfunc.loaded_model:
  artifact_path: PlainRegression_Model
  flavor: mlflow.sklearn
  run_id: bd35a5fbc2174e3e8304ffa7507d0442

In [29]:
model_name = 'ClasificadorDeVinos'
area = 'production'

model_staging_uri = "models:/{model_name}/{area}".format(model_name=model_name, area=area)
model_production = mlflow.pyfunc.load_model(model_production_uri)
model_production

mlflow.pyfunc.loaded_model:
  artifact_path: PlainRegression_Model
  flavor: mlflow.sklearn
  run_id: 5ebd8255fc00467b99b1f5689591833d

In [30]:
resultados_produccion = model_production.predict(test_x)

resultados_staging = model_staging.predict(test_x)

resultados_version = model_version.predict(test_x)

In [31]:
my_array = np.transpose(np.array([resultados_produccion, resultados_staging, resultados_version]))

df = pd.DataFrame(my_array, columns = ['Produccion','Staging','verion 1'])

df.head(100)

,Produccion,Staging,verion 1
0,5.319102,5.590129,5.610244
1,5.991788,5.987543,5.968077
2,6.118918,5.899989,5.855752
3,5.254325,5.281174,5.306589
4,5.168225,5.161197,5.169682
...,...,...,...
95,6.167877,5.944148,5.917626
96,6.040669,6.029379,6.010936
97,5.423765,5.472420,5.497752
98,5.849896,5.877173,5.901639
